## 尝试提取井的共同曲线


In [1]:
import os
from pathlib import Path

import lasio
import numpy as np


In [2]:
# 定义输入输出路径
input_folder = "../data/vertical_well_las"
output_folder = "../data/vertical_well_common_las"

# 定义基础必须曲线（按照希望的顺序）
base_curves = ["DEPT", "SP", "CAL"]

# ============================================================
# 开始处理
# ============================================================

# 创建输出文件夹
output_path = Path(output_folder)
output_path.mkdir(parents=True, exist_ok=True)

# 获取所有LAS文件
input_path = Path(input_folder)
las_files_lower = list(input_path.glob("*.las"))
las_files_upper = list(input_path.glob("*.LAS"))

# 合并并去重
las_files = list(set(las_files_lower + las_files_upper))
las_files.sort()

if not las_files:
    print(f"在 {input_folder} 中未找到LAS文件")
else:
    print(f"找到 {len(las_files)} 个LAS文件")

    # 第一步：找出所有文件的公共曲线
    all_common_curves = None
    for las_file in las_files:
        try:
            las = lasio.read(las_file, mnemonic_case="upper")
            file_curves = set([c.mnemonic for c in las.curves])

            if all_common_curves is None:
                all_common_curves = file_curves
            else:
                all_common_curves = all_common_curves.intersection(file_curves)
        except Exception as e:
            print(f"读取文件 {las_file.name} 时出错: {e}")
            continue

    if all_common_curves is None:
        print("无法读取任何LAS文件")
    else:
        print(f"\n所有文件的公共曲线: {sorted(all_common_curves)}")

        # 第二步：合并基础曲线和公共曲线，确保基础曲线在前面
        base_curves_set = set([c.upper() for c in base_curves])
        print(f"指定的基础曲线: {base_curves}")  # 保持顺序显示

        # 找出基础曲线中不在公共曲线里的
        missing_in_common = base_curves_set - all_common_curves
        if missing_in_common:
            print(f"⚠️ 注意：以下基础曲线不是所有文件都有: {sorted(missing_in_common)}")

        # 找出公共曲线中不在基础曲线里的额外曲线
        extra_common = all_common_curves - base_curves_set
        if extra_common:
            print(f"✓ 发现额外的公共曲线: {sorted(extra_common)}")

        # 最终要提取的曲线 = 基础曲线（按指定顺序）+ 其他公共曲线（排序）
        curves_to_extract = [c.upper() for c in base_curves] + sorted(extra_common)

        # 确保DEPT在第一位
        if "DEPT" in curves_to_extract:
            curves_to_extract.remove("DEPT")
            curves_to_extract.insert(0, "DEPT")
        else:
            print("⚠️ 警告：DEPT不在曲线列表中！")

        print(f"\n最终将提取的曲线（按顺序）: {curves_to_extract}")
        print(f"共 {len(curves_to_extract)} 条曲线")
        print(f"第一列曲线: {curves_to_extract[0]}")
        print("\n" + "=" * 60)

        # 第三步：处理每个LAS文件
        success_count = 0
        failed_files = []

        for las_file in las_files:
            try:
                # 读取LAS文件
                las = lasio.read(las_file, mnemonic_case="upper")

                # 获取文件中存在的曲线
                available_curves = [c.mnemonic for c in las.curves]

                # 找出实际可以提取的曲线
                actual_curves = [c for c in curves_to_extract if c in available_curves]
                missing_curves = [c for c in curves_to_extract if c not in available_curves]

                if missing_curves:
                    print(f"⚠️ 文件 {las_file.name} 缺少曲线: {missing_curves}")

                if not actual_curves:
                    print(f"✗ 文件 {las_file.name} 中没有找到任何指定的曲线，跳过")
                    failed_files.append((las_file.name, "无指定曲线"))
                    continue

                # 检查DEPT是否存在
                if "DEPT" not in actual_curves:
                    print(f"✗ 文件 {las_file.name} 缺少DEPT曲线，跳过")
                    failed_files.append((las_file.name, "缺少DEPT曲线"))
                    continue

                # 创建新的LAS对象
                new_las = lasio.LASFile()

                # 复制版本和井信息
                new_las.version = las.version
                new_las.well = las.well
                new_las.params = las.params
                new_las.other = las.other

                # 清空curves section
                new_las.curves = lasio.SectionItems()

                # 按照curves_to_extract的顺序添加曲线
                for curve_name in curves_to_extract:
                    if curve_name in available_curves:
                        curve = las.curves[curve_name]
                        new_las.append_curve(
                            mnemonic=curve.mnemonic,
                            data=curve.data,
                            unit=curve.unit,
                            descr=curve.descr,
                            value=curve.value,
                        )

                # 最终检查：确认第一列是DEPT
                if len(new_las.curves) > 0:
                    first_curve_name = new_las.curves[0].mnemonic
                    if first_curve_name != "DEPT":
                        print(f"⚠️ 警告：文件 {las_file.name} 第一列不是DEPT，而是 {first_curve_name}")

                # 保存新的LAS文件
                output_file = output_path / las_file.name
                new_las.write(str(output_file), version=2.0)

                print(
                    f"✓ 成功处理 {las_file.name}，提取了 {len(actual_curves)}/{len(curves_to_extract)} 条曲线，第一列: {new_las.curves[0].mnemonic}"
                )
                success_count += 1

            except Exception as e:
                print(f"✗ 处理文件 {las_file.name} 时出错: {e}")
                failed_files.append((las_file.name, str(e)))

        # 输出统计信息
        print("\n" + "=" * 60)
        print(f"处理完成！")
        print(f"成功: {success_count}/{len(las_files)}")

        if failed_files:
            print(f"\n失败的文件:")
            for filename, error in failed_files:
                print(f"  - {filename}: {error}")

找到 7 个LAS文件

所有文件的公共曲线: ['AMP', 'BOOL_POR', 'CN', 'DEN', 'DEPT', 'DT', 'FACIES', 'FLUIDS', 'GR', 'GR-NORM', 'GR1', 'GRINPEFA', 'INPEFA', 'LITH', 'LITH_SHOW', 'LLD1', 'MLP-YC', 'PEFA', 'PERM', 'POR', 'SAND??SHADIBI', 'SESMIC', 'SESMIC2', 'SVM-YC', 'SW', 'TWTPICKED', 'TWTPICKED2', 'VSH']
指定的基础曲线: ['DEPT', 'SP', 'CAL']
⚠️ 注意：以下基础曲线不是所有文件都有: ['CAL', 'SP']
✓ 发现额外的公共曲线: ['AMP', 'BOOL_POR', 'CN', 'DEN', 'DT', 'FACIES', 'FLUIDS', 'GR', 'GR-NORM', 'GR1', 'GRINPEFA', 'INPEFA', 'LITH', 'LITH_SHOW', 'LLD1', 'MLP-YC', 'PEFA', 'PERM', 'POR', 'SAND??SHADIBI', 'SESMIC', 'SESMIC2', 'SVM-YC', 'SW', 'TWTPICKED', 'TWTPICKED2', 'VSH']

最终将提取的曲线（按顺序）: ['DEPT', 'SP', 'CAL', 'AMP', 'BOOL_POR', 'CN', 'DEN', 'DT', 'FACIES', 'FLUIDS', 'GR', 'GR-NORM', 'GR1', 'GRINPEFA', 'INPEFA', 'LITH', 'LITH_SHOW', 'LLD1', 'MLP-YC', 'PEFA', 'PERM', 'POR', 'SAND??SHADIBI', 'SESMIC', 'SESMIC2', 'SVM-YC', 'SW', 'TWTPICKED', 'TWTPICKED2', 'VSH']
共 30 条曲线
第一列曲线: DEPT

✓ 成功处理 PH1.las，提取了 30/30 条曲线，第一列: DEPT
✓ 成功处理 PH13.las，提取了 30/30